## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
path = "/Users/ecem/Desktop/phonocardiogram/data/TV/absent"

In [4]:
waves = np.load(path + "/absent_TV_waves_10sec.npy", allow_pickle= True)


In [5]:
print(waves.shape)


(5718, 40000)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
lbs.feature.zero_crossing_rate(waves[7],frame_length= 8000, hop_length = 12000)

array([[0.023   , 0.036875, 0.034625, 0.03325 ]])

In [8]:
lbs.feature.zero_crossing_rate(waves[7], frame_length= 8000, hop_length = 12000).shape

(1, 4)

In [9]:
def zero_crossing(array, frame_length= 2048, hl = 512):
    zero_crossing_rate = np.ones(shape = (array.shape[0], 5)) #buradaki ikinci sayı hop lenght e göre değişmeli 

    for i in range(array.shape[0]):
        zero_crossing_rate[i][0:4] = lbs.feature.zero_crossing_rate(array[i],  frame_length = frame_length,
                                                                    hop_length = hl)
        zero_crossing_rate[i][4] = sum(lbs.zero_crossings(array[i]))
    return zero_crossing_rate

        

In [10]:
zcrs = zero_crossing(waves, 8000, hl = 12000) #çok uzun sürüyor, ya datayı kısaltmalıyız ya da hop lenght artmalı

In [15]:
type(zcrs[0])

numpy.ndarray

In [16]:
np.save(path + "/absent_TV_zcrs_array", zcrs, allow_pickle= True)

In [17]:
columns_zcr = []
for i in range(zcrs.shape[1]-1):
    columns_zcr.append("zcrs{}".format(i))

    
columns_zcr.append("#ofcrossing")

print(columns_zcr)

['zcrs0', 'zcrs1', 'zcrs2', 'zcrs3', '#ofcrossing']


In [18]:
zcrs[7]

array([2.3000e-02, 3.6875e-02, 3.4625e-02, 3.3250e-02, 1.4230e+03])

In [19]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[0]):
        llist = list(array[i][0:l-1].tolist())
        llist.append(array[i][l-1])
        
        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [20]:
df = dataframe_creator(zcrs, l =5,  columns = columns_zcr)

df.reset_index(drop = True, inplace = True)

In [21]:
df.head()

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing
0,0.013875,0.043500,0.040250,0.026000,1429.0
1,0.017625,0.040375,0.027375,0.031250,1306.0
2,0.013125,0.031500,0.042000,0.040625,1456.0
3,0.018750,0.043375,0.029250,0.024000,1417.0
4,0.029625,0.043750,0.066375,0.063375,2078.0


In [22]:
df.to_csv( path + "/absent_TV_zcrs.csv")

## with Özgün Ozan Nacitarhan code

In [23]:
def zcr(array):
    zcr = np.ones(shape = (array.shape[0],1))
    for i in range(array.shape[0]):
        data_zeros = np.zeros(len(array[i]))
        data_zeros[1:] = array[i][:len(array[i]) - 1]
        zcr[i][0] = (1 / (2 * len(array[i]))) * np.sum(np.abs(np.sign(array[i]) - np.sign(data_zeros)))
        
    return zcr


In [24]:
zcrs_özgün = zcr(waves)

In [25]:
zcrs_özgün.shape

(5718, 1)

In [26]:
zcrs_özgün[0]

array([0.0357125])

In [27]:
np.save(path + "/absent_TV_zcrs_özgün", zcrs_özgün, allow_pickle= True)

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import librosa.display

In [30]:
lbs.feature.chroma_stft(waves[0], sr = 4000, hop_length = 12000).shape

(12, 4)

In [31]:
# we are taking time average of each pitch values.
def chromagram_average(array, sampling_rate = 256):
    pitch = np.ones(shape = (array.shape[0], 12)) 
    
    for i in range(array.shape[0]):
        for j in range(12):
            pitch[i][j] = lbs.feature.chroma_stft(np.array(array[i]), sr= sampling_rate, hop_length = 12000)[j].mean()
    return pitch

In [32]:
chro = chromagram_average(waves, sampling_rate = 4000) # bu da çok uzun sürüyor.

In [33]:
type(chro[0])

numpy.ndarray

In [34]:
chro.shape

(5718, 12)

In [35]:
np.save(path + "/absent_TV_chromogram_array", chro, allow_pickle= True)

In [36]:
print(chro.shape)

(5718, 12)


In [37]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [38]:
col_chro = []
for i in range(chro.shape[1]):
    col_chro.append(lst[i])
    
print(col_chro)
print(len(col_chro))

['C', 'C♯', 'D', 'D♯', 'E', 'F', 'F♯', 'G', 'G♯', 'A', 'A♯', 'B']
12


In [39]:
ch_df = dataframe_creator(chro,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.774697,0.752658,0.624351,0.614751,0.505404,0.562265,0.691305,0.708033,0.747035,0.612049,0.673493,0.70302
1,0.528984,0.569655,0.631345,0.469145,0.581654,0.628005,0.609931,0.617767,0.658815,0.788469,0.638231,0.490605
2,0.677536,0.571465,0.455335,0.636636,0.458969,0.446903,0.491728,0.43595,0.524053,0.692241,0.771804,0.758902
3,0.590559,0.798237,0.52909,0.415183,0.528545,0.482318,0.430858,0.417283,0.437482,0.548379,0.541304,0.498784
4,0.740675,0.729426,0.650502,0.714158,0.789993,0.751961,0.606954,0.669602,0.731011,0.677369,0.536382,0.584563
...,...,...,...,...,...,...,...,...,...,...,...,...
5713,0.453768,0.511931,0.688536,0.518481,0.551608,0.681667,0.859559,0.614479,0.596402,0.697829,0.645281,0.441324
5714,0.223454,0.246843,0.28603,0.344106,0.229377,0.500183,0.555623,0.246648,0.229102,0.402806,0.279365,0.307494
5715,0.324664,0.649997,0.461658,0.378426,0.33382,0.510383,0.477204,0.294466,0.474113,0.635614,0.668764,0.531328
5716,0.447829,0.72264,0.788603,0.556185,0.427288,0.690504,0.552843,0.399766,0.332237,0.287436,0.42689,0.399635


In [40]:
ch_df.to_csv(path + "/absent_TV_chromogram.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [41]:
def tempo_average(array): # 192,79
    means= []
    for i in range(array.shape[0]): #192
        means.append(array[i].mean())
    return np.array(means) # 192
        

In [42]:
def tempogram(array = 0, hop_lenght = 12000, sampling_rate = 256):
    
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sampling_rate, 
                                        lag =3)

    times = librosa.times_like(oenv, sr=sampling_rate, hop_length=12000)

    tempogram = librosa.feature.tempogram(onset_envelope = oenv, sr = sampling_rate,
                                              hop_length = hop_lenght, win_length = 192 )
    # Estimate the global tempo for display purposes
    #tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sampling_rate,
                                   #hop_length=hop_length)[0]
    return tempogram

In [43]:
tempogram(waves[0]).shape

(192, 79)

In [44]:
tempogram(waves[0])[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [45]:
tempogram(waves[0])[1]

array([0.8043611 , 0.80387406, 0.80338417, 0.80289217, 0.80239871,
       0.80190442, 0.80140988, 0.80091561, 0.8004221 , 0.7999298 ,
       0.79943913, 0.79895047, 0.79846417, 0.79798056, 0.79749994,
       0.7970226 , 0.7965488 , 0.79607877, 0.79561275, 0.79515094,
       0.79469355, 0.79424076, 0.79379275, 0.79334969, 0.79291174,
       0.79247906, 0.79205181, 0.79163012, 0.79121414, 0.79080403,
       0.79039993, 0.79000197, 0.78961032, 0.78922512, 0.78884652,
       0.78847469, 0.78810978, 0.78775198, 0.78740145, 0.78705839,
       0.786723  , 0.78639547, 0.78607604, 0.78576492, 0.78546236,
       0.78516863, 0.784884  , 0.78460876, 0.78434322, 0.7840877 ,
       0.78384257, 0.78360819, 0.78338496, 0.78317331, 0.78297367,
       0.78278654, 0.78261241, 0.78245184, 0.7823054 , 0.7821737 ,
       0.7820574 , 0.78195719, 0.78187383, 0.78180809, 0.78176082,
       0.78173291, 0.78172532, 0.78173906, 0.78177521, 0.78183492,
       0.78191941, 0.78202999, 0.78216803, 0.78233502, 0.78253

In [46]:
tempo_average(tempogram(waves[0]))

array([ 1.00000000e+00,  7.89280253e-01,  4.49616051e-01,  2.21368033e-01,
        1.30775800e-01,  9.80726553e-02,  1.05266714e-01,  1.51666437e-01,
        2.51141114e-01,  3.69657473e-01,  4.84736698e-01,  5.46325512e-01,
        5.01062008e-01,  3.49767069e-01,  2.07547145e-01,  1.34799538e-01,
        1.11587554e-01,  9.78534442e-02,  1.09711555e-01,  1.60357465e-01,
        2.76904079e-01,  4.52404942e-01,  5.91524878e-01,  5.65543427e-01,
        3.93683941e-01,  2.06785105e-01,  1.00987381e-01,  8.39411039e-02,
        9.52436375e-02,  1.09777802e-01,  1.48008206e-01,  2.17351582e-01,
        3.03280822e-01,  3.40376030e-01,  3.21362998e-01,  2.67205206e-01,
        1.97394007e-01,  1.31212459e-01,  9.70357672e-02,  8.21447951e-02,
        8.17802786e-02,  1.00844825e-01,  1.39599025e-01,  2.05209543e-01,
        2.56221610e-01,  2.38765863e-01,  1.71433904e-01,  9.92135991e-02,
        6.37648549e-02,  5.61266669e-02,  5.78066402e-02,  6.59663867e-02,
        7.87572987e-02,  

# tempogram(waves[0])[0]

In [47]:
def temp_values(array):
    final = np.ones(shape = (array.shape[0], 192))
    for i in range(array.shape[0]): #8bin civarı
        
        _x = tempogram(array[i]) #1,4000   #192,79
        #print(_x.shape)
        final[i] = tempo_average(_x) #192
        
        
    return final

In [48]:
temp = temp_values(waves) #çok uzun sürüyor 


In [49]:
type(temp)

numpy.ndarray

In [50]:
np.save(path + "/absent_TV_tempgram_array", temp, allow_pickle= True)

In [51]:
temp.shape

(5718, 192)

In [52]:
columns_t = []
for i in range(temp.shape[1]):
    columns_t.append("tempogram_{}".format(i))
print(columns_t)

['tempogram_0', 'tempogram_1', 'tempogram_2', 'tempogram_3', 'tempogram_4', 'tempogram_5', 'tempogram_6', 'tempogram_7', 'tempogram_8', 'tempogram_9', 'tempogram_10', 'tempogram_11', 'tempogram_12', 'tempogram_13', 'tempogram_14', 'tempogram_15', 'tempogram_16', 'tempogram_17', 'tempogram_18', 'tempogram_19', 'tempogram_20', 'tempogram_21', 'tempogram_22', 'tempogram_23', 'tempogram_24', 'tempogram_25', 'tempogram_26', 'tempogram_27', 'tempogram_28', 'tempogram_29', 'tempogram_30', 'tempogram_31', 'tempogram_32', 'tempogram_33', 'tempogram_34', 'tempogram_35', 'tempogram_36', 'tempogram_37', 'tempogram_38', 'tempogram_39', 'tempogram_40', 'tempogram_41', 'tempogram_42', 'tempogram_43', 'tempogram_44', 'tempogram_45', 'tempogram_46', 'tempogram_47', 'tempogram_48', 'tempogram_49', 'tempogram_50', 'tempogram_51', 'tempogram_52', 'tempogram_53', 'tempogram_54', 'tempogram_55', 'tempogram_56', 'tempogram_57', 'tempogram_58', 'tempogram_59', 'tempogram_60', 'tempogram_61', 'tempogram_62', '

In [53]:
len(columns_t)

192

In [54]:
df_t = dataframe_creator(temp,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 192)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_0,tempogram_1,tempogram_2,tempogram_3,tempogram_4,tempogram_5,tempogram_6,tempogram_7,tempogram_8,tempogram_9,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,1.0,0.78928,0.449616,0.221368,0.130776,0.098073,0.105267,0.151666,0.251141,0.369657,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.797984,0.482743,0.307028,0.283363,0.349212,0.463909,0.559565,0.563266,0.496564,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,1.0,0.887349,0.718716,0.605566,0.548434,0.512543,0.468736,0.438265,0.460934,0.504058,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.800527,0.475701,0.308435,0.340022,0.463117,0.521938,0.441395,0.310173,0.293569,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,1.0,0.795149,0.463031,0.247617,0.153149,0.109321,0.1169,0.208411,0.381928,0.570263,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5713,1.0,0.784318,0.437389,0.222279,0.187637,0.245674,0.287423,0.266299,0.211437,0.200573,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5714,1.0,0.853429,0.67926,0.611528,0.625351,0.621929,0.589734,0.586242,0.567186,0.555705,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
5715,1.0,0.759845,0.475647,0.307214,0.287388,0.366127,0.518031,0.588331,0.546471,0.457722,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
5716,1.0,0.803923,0.498382,0.343497,0.340038,0.395419,0.455638,0.495009,0.565344,0.61335,...,-0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [55]:
df_t['tempogram_188'][10]

1.5110125434843798e-18

In [56]:
df_t.to_csv(path + '/absent_TV_tempogram.csv')

In [57]:
df_t.isin([0]).sum(axis=0)

tempogram_0      0
tempogram_1      0
tempogram_2      0
tempogram_3      0
tempogram_4      0
                ..
tempogram_187    0
tempogram_188    0
tempogram_189    0
tempogram_190    0
tempogram_191    0
Length: 192, dtype: int64

In [58]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.013875,0.043500,0.040250,0.026000,1429.0,0.774697,0.752658,0.624351,0.614751,0.505404,0.562265,0.691305,0.708033,0.747035,0.612049,0.673493,0.70302
1,0.017625,0.040375,0.027375,0.031250,1306.0,0.528984,0.569655,0.631345,0.469145,0.581654,0.628005,0.609931,0.617767,0.658815,0.788469,0.638231,0.490605
2,0.013125,0.031500,0.042000,0.040625,1456.0,0.677536,0.571465,0.455335,0.636636,0.458969,0.446903,0.491728,0.43595,0.524053,0.692241,0.771804,0.758902
3,0.018750,0.043375,0.029250,0.024000,1417.0,0.590559,0.798237,0.52909,0.415183,0.528545,0.482318,0.430858,0.417283,0.437482,0.548379,0.541304,0.498784
4,0.029625,0.043750,0.066375,0.063375,2078.0,0.740675,0.729426,0.650502,0.714158,0.789993,0.751961,0.606954,0.669602,0.731011,0.677369,0.536382,0.584563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5713,0.033500,0.049500,0.061125,0.056250,2220.0,0.453768,0.511931,0.688536,0.518481,0.551608,0.681667,0.859559,0.614479,0.596402,0.697829,0.645281,0.441324
5714,0.033625,0.065000,0.075750,0.079125,2850.0,0.223454,0.246843,0.28603,0.344106,0.229377,0.500183,0.555623,0.246648,0.229102,0.402806,0.279365,0.307494
5715,0.024750,0.095375,0.062250,0.052625,2610.0,0.324664,0.649997,0.461658,0.378426,0.33382,0.510383,0.477204,0.294466,0.474113,0.635614,0.668764,0.531328
5716,0.027750,0.046250,0.081375,0.073750,2477.0,0.447829,0.72264,0.788603,0.556185,0.427288,0.690504,0.552843,0.399766,0.332237,0.287436,0.42689,0.399635


In [59]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,0.013875,0.043500,0.040250,0.026000,1429.0,0.774697,0.752658,0.624351,0.614751,0.505404,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.017625,0.040375,0.027375,0.031250,1306.0,0.528984,0.569655,0.631345,0.469145,0.581654,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.013125,0.031500,0.042000,0.040625,1456.0,0.677536,0.571465,0.455335,0.636636,0.458969,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.018750,0.043375,0.029250,0.024000,1417.0,0.590559,0.798237,0.52909,0.415183,0.528545,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.029625,0.043750,0.066375,0.063375,2078.0,0.740675,0.729426,0.650502,0.714158,0.789993,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5713,0.033500,0.049500,0.061125,0.056250,2220.0,0.453768,0.511931,0.688536,0.518481,0.551608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5714,0.033625,0.065000,0.075750,0.079125,2850.0,0.223454,0.246843,0.28603,0.344106,0.229377,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
5715,0.024750,0.095375,0.062250,0.052625,2610.0,0.324664,0.649997,0.461658,0.378426,0.33382,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
5716,0.027750,0.046250,0.081375,0.073750,2477.0,0.447829,0.72264,0.788603,0.556185,0.427288,...,-0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [60]:
final_df.to_csv(path + '/absent_TV_final_sp.csv')